#Create combined pd dataframe
Retrieve both gt for each input benchmark and the results for each model
#Resulting Format
dataset | dataset idx | ground_truth | (prompt/category?) | [ model | modelResult ]
using long-format dataFrame, where each model result is its separate row

In [7]:
import warnings
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# ─── Project directories ───────────────────────────────────────────────────────
BASE_DIR       = Path('/cluster/project/sachan/pmlr/grounding-vlms')
EVAL_DIR       = Path(BASE_DIR / 'eval')
ANALYSIS_DIR   = Path(EVAL_DIR / 'analysis')
DATA_DIR       = Path(EVAL_DIR / 'data')
RESULTS_DIR    = Path(EVAL_DIR / 'valid_results')
# Ensure analysis folder exists
ANALYSIS_DIR.mkdir(parents=True, exist_ok=True)

In [8]:
import warnings
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Create combined dataframe with error handling and validation
unified = pd.DataFrame()
dataset_counts = {}
model_counts = {}

# Track processing statistics
datasets_processed = 0
datasets_skipped = 0
models_processed = 0

for data_path in DATA_DIR.glob('*_dataset.csv'):
    try:
        dataset_name = data_path.stem.replace('_dataset', '')
        print(f"Processing dataset: {dataset_name}")
        
        # Load dataset with error handling
        try:
            # First check what columns are available in the dataset
            all_columns = pd.read_csv(data_path, nrows=0).columns.tolist()
            
            # Define the columns we want to extract
            required_columns = ['prompt', 'truth', 'label']
            
            # Handle file_name/image_url column variation
            file_column = None
            if 'file_name' in all_columns:
                file_column = 'file_name'
            elif 'image_url' in all_columns:
                file_column = 'image_url'
            else:
                print(f"Warning: Neither 'file_name' nor 'image_url' found in {dataset_name}")
                datasets_skipped += 1
                continue
            
            columns_to_extract = [file_column] + required_columns
            
            # Load the dataset with the correct columns
            df = pd.read_csv(data_path, usecols=columns_to_extract)
            
            # Check if all required columns exist
            missing_columns = [col for col in required_columns if col not in df.columns]
            if missing_columns:
                print(f"Warning: Missing columns in {dataset_name}: {missing_columns}")
                datasets_skipped += 1
                continue
                
            # Rename the file column consistently to file_name
            if file_column != 'file_name':
                df = df.rename(columns={file_column: 'file_name'})
            
            # Create and validate index
            df = df.reset_index(drop=False).rename(columns={'index': 'idx'})
            
            # Check for duplicate indices
            if df['idx'].duplicated().any():
                print(f"Warning: Duplicate indices found in {dataset_name}. Creating new unique indices.")
                df = df.reset_index(drop=True)
                df['idx'] = df.index
            
            df = df.astype({
                'idx': int,
                'file_name': str,
                'prompt': str,
                'truth': int,
                'label': str
            }, errors='raise')  # Catch type conversion errors
            
            df['dataset'] = dataset_name
            dataset_counts[dataset_name] = len(df)
            
        except Exception as e:
            print(f"Error loading dataset {dataset_name}: {str(e)}")
            datasets_skipped += 1
            continue
        
        # Process model results
        model_results = []
        dataset_models_processed = 0
        
        for model_dir in sorted(RESULTS_DIR.iterdir()):
            if not model_dir.is_dir():
                continue
                
            resultFile = model_dir / f'{dataset_name}_results.csv'
            if not resultFile.exists():
                print(f"Information: Results for {dataset_name} not found in {model_dir.name}")
                continue
                
            try:
                rd_df = pd.read_csv(resultFile, usecols=['idx', 'result'])
                
                # Validate model results
                if rd_df.empty:
                    print(f"Warning: Empty results for {model_dir.name} on {dataset_name}")
                    continue
                    
                rd_df['model'] = model_dir.stem
                rd_df = rd_df.astype({
                    'idx': int,
                    'result': int,
                    'model': str
                })
                
                # Check if model indices match dataset indices
                missing_indices = set(df['idx']) - set(rd_df['idx'])
                if missing_indices:
                    print(f"Warning: Model {model_dir.name} is missing {len(missing_indices)} indices for {dataset_name}")
                rd_df = rd_df[rd_df['result']!=-1]  # Filter out invalid results
                
                model_results.append(rd_df)
                dataset_models_processed += 1
                model_counts[model_dir.stem] = model_counts.get(model_dir.stem, 0) + 1
                
            except Exception as e:
                print(f"Error processing {model_dir.name} for {dataset_name}: {str(e)}")
                continue
        
        if model_results:
            # Combine all model results
            all_results = pd.concat(model_results, ignore_index=True)
            
            # Merge with dataset info with appropriate error handling
            try:
                final_df = df.merge(all_results.rename(columns={'result': 'model_result'}), 
                                    on='idx', how='left', validate='one_to_many')
                
                # Add to unified dataframe
                unified = pd.concat([unified, final_df], ignore_index=True)
                models_processed += dataset_models_processed
                datasets_processed += 1
                
            except pd.errors.MergeError as e:
                print(f"Merge error for {dataset_name}: {str(e)}")
        else:
            print(f"No valid model results found for {dataset_name}")
    
    except Exception as e:
        print(f"Unexpected error processing {data_path}: {str(e)}")

# Check if we have data to save
if not unified.empty:
    # Ensure output directory exists
    RESULTS_DIR.mkdir(parents=True, exist_ok=True)
    
    # Create output path with safeguards
    output_path = RESULTS_DIR / "combinedResults.csv"
    
    # Backup existing file if it exists
    if output_path.exists():
        backup_path = RESULTS_DIR / f"combinedResults_backup.csv"
        os.rename(output_path, backup_path)
        print(f"Existing file backed up to: {backup_path}")
    
    # Save the unified results
    unified.to_csv(output_path, index=False)
    print(f"Combined results saved to: {output_path}")
    
    # Display basic statistics
    print(f"\nSummary Statistics:")
    print(f"Combined dataset size: {len(unified)} rows")
    print(f"Datasets processed successfully: {datasets_processed} of {datasets_processed + datasets_skipped}")
    print(f"Models evaluated: {unified['model'].nunique()} unique models")
    print(f"Total dataset-model combinations: {len(unified)}")
    
    # Show dataset distribution
    print("\nRows per dataset:")
    dataset_distribution = unified.groupby('dataset').size()
    for dataset, count in dataset_distribution.items():
        print(f"  - {dataset}: {count} rows")
        
    # Show model distribution
    print("\nDatasets evaluated by each model:")
    for model, count in model_counts.items():
        print(f"  - {model}: {count} datasets")
else:
    print("Error: No data was processed successfully. No output file created.")


Processing dataset: FSC-147
Processing dataset: GeckoNum
Information: Results for GeckoNum not found in gemini-2.5-pro-preview-05-06
Information: Results for GeckoNum not found in gemma-3-12b-it
Information: Results for GeckoNum not found in gemma-3-27b-it
Information: Results for GeckoNum not found in gemma-3-4b-it
Processing dataset: PixMo_Count
Information: Results for PixMo_Count not found in GPT-4.1
Information: Results for PixMo_Count not found in claude-3.5-haiku
Information: Results for PixMo_Count not found in gemini-2.5-flash-preview-04-17
Information: Results for PixMo_Count not found in gemini-2.5-pro-preview-05-06
Information: Results for PixMo_Count not found in gemma-3-12b-it
Information: Results for PixMo_Count not found in gemma-3-27b-it
Information: Results for PixMo_Count not found in gemma-3-4b-it
Information: Results for PixMo_Count not found in grok-2-vision
Information: Results for PixMo_Count not found in o4-mini
No valid model results found for PixMo_Count
Proc

In [9]:
# Set display options to show all columns in one row
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)        # Set wide display width
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping

print(unified.head())

   idx file_name                                    prompt  truth       label  dataset  model_result           model
0    0     2.jpg  How many sea shells are in this picture?      8  sea shells  FSC-147           8.0           GPT-4
1    0     2.jpg  How many sea shells are in this picture?      8  sea shells  FSC-147           7.0        claude-3
2    0     2.jpg  How many sea shells are in this picture?      8  sea shells  FSC-147           8.0        gemini-2
3    0     2.jpg  How many sea shells are in this picture?      8  sea shells  FSC-147           7.0        gemini-2
4    0     2.jpg  How many sea shells are in this picture?      8  sea shells  FSC-147           8.0  gemma-3-12b-it


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def plot_model_accuracies_by_bin(csv_path, bin_width, value_range, log_scale=False, minData=3, output_path='accuracy_plot.pdf'):
    # Unpack the value_range tuple into min_value and max_value
    min_value, max_value = value_range
    
    # Load data
    df = pd.read_csv(csv_path)

    # Ensure numeric types for comparison
    df['truth'] = pd.to_numeric(df['truth'], errors='coerce')
    df['model_result'] = pd.to_numeric(df['model_result'], errors='coerce')

    # Filter out rows with NaNs in numeric columns
    df = df.dropna(subset=['truth', 'model_result'])

    # Filter rows based on the specified value range
    df = df[(df['truth'] >= min_value) & (df['truth'] <= max_value)]

    # Create bin edges based on the range
    bins = np.arange(min_value, max_value + bin_width, bin_width)
    df['bin'] = pd.cut(df['truth'], bins=bins, include_lowest=True)

    # Determine model accuracies in each bin
    df['correct'] = df['truth'] == df['model_result']
    # Fix deprecated observed parameter warning
    grouped = df.groupby(['model', 'bin'], observed=False)['correct'].agg(['mean', 'count']).reset_index()

    # Pivot the data to plot by model
    pivot_mean = grouped.pivot(index='bin', columns='model', values='mean')
    pivot_count = grouped.pivot(index='bin', columns='model', values='count')

    # Calculate bin centers for plotting (keep this for the actual plot positioning)
    bin_centers = [(interval.left + interval.right) / 2 for interval in pivot_mean.index]
    
    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))
    # Fix deprecated get_cmap warning
    cmap = plt.colormaps['tab10']
    num_models = len(pivot_mean.columns)

    for i, model in enumerate(pivot_mean.columns):
        # Copy the data and set NaN for bins with insufficient data
        plot_data = pivot_mean[model].copy()
        mask = pivot_count[model] < minData
        plot_data[mask] = np.nan
        
        # Plot the data - matplotlib will automatically skip NaN values but connect the line
        ax.plot(bin_centers, plot_data,
                label=model,
                marker='o',
                markersize=2,
                linewidth=1,
                color=cmap(i % cmap.N))

    ax.set_xlabel('Truth Value Bins')
    ax.set_ylabel('Accuracy')
    ax.set_title(f'Model Accuracy by Truth Value Bin ({min_value}-{max_value}), Bin Width: {bin_width}')
    ax.legend()
    ax.grid(True)

    if log_scale:
        ax.set_yscale('log')

    # Adjust x-axis ticks and labels (every 5th bin)
    tick_indices = np.arange(0, len(bin_centers), 5)
    ax.set_xticks([bin_centers[i] for i in tick_indices if i < len(bin_centers)])
    # Format labels to show inclusive bounds [lower, upper]
    ax.set_xticklabels([f"[{interval.left:.0f},{(interval.right-1):.0f}]" 
                        for interval in [pivot_mean.index[i] for i in tick_indices if i < len(bin_centers)]],
                        rotation=45, ha='right')

    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

plot_model_accuracies_by_bin(
    csv_path=RESULTS_DIR / 'combinedResults.csv',
    bin_width=2,
    value_range=(0, 100),
    log_scale=False,
    minData=10,
    output_path=ANALYSIS_DIR / 'plots' / 'accuracy_plot.pdf'
)



In [11]:
df = pd.read_csv(RESULTS_DIR / 'combinedResults.csv')

# Group by truth, label, AND model
grouped = df.groupby(['truth', 'label', 'model'])

# Store results for sorting
results = []

for name, group in grouped:
    truth, label, model_name = name
    
    # Only include groups with multiple files (same model on different files with same truth/label)
    if group['file_name'].nunique() > 1:
        min_val = group['model_result'].min()
        max_val = group['model_result'].max()
        mean_val = group['model_result'].mean()
        std_val = group['model_result'].std()
        
        # Calculate max deviation from truth
        max_diff = abs(max_val - truth)
        min_diff = abs(min_val - truth)
        max_deviation = max(max_diff, min_diff)
        
        # Calculate variance across files
        variance = group['model_result'].var()
        
        results.append({
            'truth': truth,
            'label': label,
            'model': model_name,
            'max_deviation': max_deviation,
            'variance': variance,
            'min_val': min_val,
            'max_val': max_val,
            'mean_val': mean_val,
            'std_val': std_val,
            'file_count': group['file_name'].nunique(),
            'group_data': group
        })

# Sort by max deviation (highest first)
results.sort(key=lambda x: x['max_deviation'], reverse=True)

# Print the top 10 groups with highest deviation
print("Top 10 truth-label-model combinations with highest deviation:")
for i, item in enumerate(results[:10]):
    print(f"\nGroup {i+1}: Max Deviation = {item['max_deviation']}")
    print(f"Truth: {item['truth']}, Label: {item['label']}, Model: {item['model']}")
    print(f"Files: {item['file_count']} unique files")
    print(f"Min: {item['min_val']}, Max: {item['max_val']}, Mean: {item['mean_val']:.2f}, Std: {item['std_val']:.2f}")
    
    # Show sample results grouped by file
    print("\nResults by file:")
    for file_name, file_group in item['group_data'].groupby('file_name'):
        for _, row in file_group.iterrows():
            deviation = abs(row['model_result'] - item['truth'])
            print(f"  File: {file_name}, Result: {row['model_result']} (deviation: {deviation})")

Top 10 truth-label-model combinations with highest deviation:

Group 1: Max Deviation = 1299999.0
Truth: 1, Label: types, Model: o4-mini
Files: 21 unique files
Min: 0.0, Max: 1300000.0, Mean: 61905.81, Std: 283683.02

Results by file:
  File: VG_100K/2324682.jpg, Result: 1.0 (deviation: 0.0)
  File: train2014/COCO_train2014_000000036059.jpg, Result: 1.0 (deviation: 0.0)
  File: train2014/COCO_train2014_000000038046.jpg, Result: 2.0 (deviation: 1.0)
  File: train2014/COCO_train2014_000000075821.jpg, Result: 1.0 (deviation: 0.0)
  File: train2014/COCO_train2014_000000151892.jpg, Result: 1.0 (deviation: 0.0)
  File: train2014/COCO_train2014_000000166261.jpg, Result: 1.0 (deviation: 0.0)
  File: train2014/COCO_train2014_000000167453.jpg, Result: 1.0 (deviation: 0.0)
  File: train2014/COCO_train2014_000000179454.jpg, Result: 1.0 (deviation: 0.0)
  File: train2014/COCO_train2014_000000202531.jpg, Result: 1300000.0 (deviation: 1299999.0)
  File: train2014/COCO_train2014_000000224178.jpg, Resu

In [ ]:
# Define minimum data threshold
minData = 5  # Adjust this value as needed

# First calculate counts for each truth-label combination (across all models)
truth_label_counts = df.groupby(['truth', 'label']).size().reset_index(name='count')

# Filter to only include combinations with at least minData values
valid_combinations = truth_label_counts[truth_label_counts['count'] >= minData]

# Apply the filter to the original dataframe
df_filtered = df.merge(valid_combinations[['truth', 'label']], on=['truth', 'label'])

# Group by truth and label (not model) to get aggregate metrics
grouped_by_truth_label = df_filtered.groupby(['truth', 'label'])

# Calculate accuracy metrics across all models for each truth-label combination
label_accuracy = grouped_by_truth_label.apply(lambda g: pd.Series({
    'count': len(g),
    'exact_match_rate': (g['truth'] == g['model_result']).mean(),
    'within_one_rate': (abs(g['truth'] - g['model_result']) <= 1).mean(),
    'mean_abs_error': abs(g['truth'] - g['model_result']).mean(),
    'std_dev': g['model_result'].std(),
    'num_models': g['model'].nunique()  # Count unique models in this group
})).reset_index()

# Sort by truth value for easier analysis
label_accuracy = label_accuracy.sort_values(['truth', 'label'])

print(f"Accuracy metrics aggregated by truth-label combinations (with minimum {minData} samples):")
print(label_accuracy[['truth', 'label', 'count', 'num_models', 'exact_match_rate', 'within_one_rate', 'mean_abs_error', 'std_dev']])

# Optionally save the results to CSV
output_path = ANALYSIS_DIR / "truth_label_accuracy.csv"
label_accuracy.to_csv(output_path, index=False)
print(f"\nResults saved to: {output_path}")

Per label accuracy numbers grouped by truth and model (with minimum 5 samples):
       truth    label           model  count  exact_match_rate  within_one_rate  mean_abs_error  std_dev
0          0      CDs           GPT-4    1.0               1.0              1.0             0.0      0.0
1          0      CDs        claude-3    1.0               1.0              1.0             0.0      0.0
2          0      CDs        gemini-2    1.0               1.0              1.0             0.0      0.0
3          0      CDs  gemma-3-12b-it    1.0               1.0              1.0             0.0      0.0
4          0      CDs  gemma-3-27b-it    1.0               1.0              1.0             0.0      0.0
...      ...      ...             ...    ...               ...              ...             ...      ...
50228   3701  markers  gemma-3-12b-it    1.0               0.0              0.0          3438.0      0.0
50229   3701  markers  gemma-3-27b-it    1.0               0.0              0.0 

/tmp/ipykernel_234761/264667729.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  label_accuracy = grouped_by_label.apply(lambda g: pd.Series({
/tmp/ipykernel_234761/264667729.py:40: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pivot_exact_match = label_accuracy.pivot_table(
/tmp/ipykernel_234761/264667729.py:51: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) ins